# Model evaluation 

## Setup

In [1]:
import os
from collections import defaultdict
import pandas as pd
from matplotlib import pyplot as plt 
import seaborn as sns
from omegaconf import DictConfig, OmegaConf


output_dir = r"/home/yiting/Documents/GitHub/lightning-pose/outputs"
# Pose PCA loss
model_dirs = ["2024-08-12/18-04-44", "2024-08-09/15-12-36"]
model_names = ["no_pose_pca", "pose_pca"]
# model_dir = r"2024-08-09/15-12-36"
# model_name = "PosePCA"
save_dir = r"/home/yiting/Documents/LP_projects/model_eval/pose_pca_loss"


### Preparation for multiview datasets (optional)

In [ ]:
# Concatenate data across camera views

# Get view names
view_names = cfg.data.view_names
df_all = pd.DataFrame()
for view in view_names:
    file_path = os.path.join(output_dir, model_dir, "predictions_" + view +".csv")
    df = pd.read_csv(file_path, header=[0, 1, 2], index_col=0)
    df_all = pd.concat([df_all, df])
df_all.to_csv(os.path.join(output_dir, model_dir, "predictions.csv"))

df_error_all = pd.DataFrame()
for view in view_names:
    file_path = os.path.join(output_dir, model_dir, "predictions_" + view +"_pixel_error.csv")
    df = pd.read_csv(file_path, index_col=0)
    df_error_all = pd.concat([df_error_all, df])
df_error_all.to_csv(os.path.join(output_dir, model_dir, "predictions_pixel_error.csv"))

## Labeled Frame Diagnostics

### Multipel model comparison

In [2]:
# Load data 
dframes_metrics = defaultdict(dict)
df_data_types = defaultdict(dict)
for model_dir, model_name in zip(model_dirs, model_names):
    # Load hydra configuration file
    cfg = OmegaConf.load(os.path.join(output_dir, model_dir, "config.yaml"))
    # Load prediction files
    prediction_files_all = [filename for filename in os.listdir(os.path.join(output_dir,model_dir)) 
                    if filename.endswith('.csv')]
    # Select predictions.csv and predictions_pixel_error.csv (removing camera view files)
    prediction_files = [filename for filename in prediction_files_all if 'cam' not in str(filename)]
    # Get keypoint names
    keypoint_names = cfg.data.keypoint_names
    # Load evaluation data
    for model_pred_file in prediction_files:
        model_pred_file_path = os.path.join(output_dir, model_dir, model_pred_file)
        model_pred_file_name, ext = os.path.splitext(model_pred_file)
        if "pca" in str(model_pred_file) or "temporal" in str(model_pred_file) or "pixel" in str(model_pred_file):
            dframe = pd.read_csv(model_pred_file_path, index_col=None)
            dframes_metrics[model_name][str(model_pred_file_name)] = dframe
        else:
            dframe = pd.read_csv(model_pred_file_path, header=[1, 2], index_col=0)
            new_df = pd.DataFrame()
            for kp in keypoint_names:
                new_df[kp] = dframe[kp]['likelihood']
            new_df['set'] = dframe.iloc[:,-1]
            dframes_metrics[model_name]["confidence"] = new_df
        df_data_types[model_name] = dframe.iloc[:, -1].unique()
    

In [ ]:
# Plot
metric_names = dframes_metrics[model_names[0]].keys()
data_types = df_data_types[model_names[0]]

for me, metric in enumerate(metric_names):
    fig, axs = plt.subplots(len(data_types), len(model_names), figsize=(5*len(model_names), 5*len(data_types)), sharey=True)
    for mo, model_name in enumerate(model_names):
        df = dframes_metrics[model_name][metric]
        
        for d, data_type in enumerate(data_types):
            df_data_type = df[df['set'] == data_type]
            sns.boxplot(data = df_data_type, ax=axs[d, mo])
            # Setup title, labels, axes
            axs[d, mo].spines['top'].set_visible(False)
            axs[d, mo].spines['right'].set_visible(False)
            axs[d, mo].tick_params(axis='x', rotation=45)  # Rotate x-tick labels
            axs[d, mo].set_ylabel(data_type + "\n" + metric)
            axs[d, mo].set_ylim(0, 50)
            if d == 0:
                axs[d, mo].set_title(model_name)
    # Adjust layout to prevent overlap
    plt.tight_layout()

    # Save the figure
    plt.savefig(os.path.join(save_dir, model_name + "_" + metric + ".png")) 

### Single model

In [ ]:
prediction_files_all = [filename for filename in os.listdir(os.path.join(output_dir,model_dir)) 
                    if filename.endswith('.csv')]
# Select predictions.csv and predictions_pixel_error.csv (removing camera view files)
prediction_files = [filename for filename in prediction_files_all if 'cam' not in str(filename)]
# Get keypoint names
keypoint_names = cfg.data.keypoint_names
# Load evaluation data
dframes_metrics = defaultdict(dict)
for model_pred_file in prediction_files:
    model_pred_file_path = os.path.join(output_dir, model_dir, model_pred_file)
    model_pred_file_name, ext = os.path.splitext(model_pred_file)
    if "pca" in str(model_pred_file) or "temporal" in str(model_pred_file) or "pixel" in str(model_pred_file):
        dframe = pd.read_csv(model_pred_file_path, index_col=None)
        dframes_metrics[str(model_pred_file_name)] = dframe
    else:
        dframe = pd.read_csv(model_pred_file_path, header=[1, 2], index_col=0)
        new_df = pd.DataFrame()
        for kp in keypoint_names:
            new_df[kp] = dframe[kp]['likelihood']
        new_df['set'] = dframe.iloc[:,-1]
        dframes_metrics["confidence"] = new_df
    data_types = dframe.iloc[:, -1].unique()

In [ ]:
def plot_metric_per_keypoint(df, metric, model_name, data_types):

    fig, axs = plt.subplots(len(data_types), 1, figsize=(8, 5*len(data_types)))
    for d, data_type in enumerate(data_types):
        df_data_type = df[df['set'] == data_type]
        sns.boxplot(data = df_data_type, ax=axs[d])
        # Setup title, labels, axes
        axs[d].spines['top'].set_visible(False)
        axs[d].spines['right'].set_visible(False)
        axs[d].set_ylabel(metric)
        # axs[d].set_xlabel("keypoint")
        axs[d].tick_params(axis='x', rotation=45)  # Rotate x-tick labels
        axs[d].set_title(data_type)
        # axs[d].set_ylim()

    # Add a super title
    fig.suptitle(model_name, fontsize=16)

    # Adjust layout
    plt.tight_layout(rect=[0, 0, 1, 0.98])  # Adjust the rect to make room for the super title
    
    return fig, axs